In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html
import os

### 定数

In [11]:
REGION_NAME = 'ap-northeast-1'

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")

DATASET_GROUP_NAME = "word_suggest_recommendation"
INTERACTIONS_FILENAME = "search_log.csv" # 要修正
ROLE_NAME = "PersonalizeRoleAmazonPantryAwsSimilarItems"

BOUGHT_TOGETHER_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together"
VIEWED_ALSO_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed"

# レシピ名
SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-sims"
# SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together"

SCHEMA_NAME = "suggest-word-1"

### データ読み込み

In [4]:
search_log_df = pd.read_csv(INTERACTIONS_FILENAME)

search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,user_001,coffee,1704067200,search
1,user_001,tea,1704067200,search
2,user_001,milk,1704067200,search
3,user_002,bread,1704153600,search
4,user_002,cheese,1704153600,search


In [5]:
search_log_df.describe(include='all')

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
count,4913,4913,4.913000e+03,4913
unique,30,85,NaN,1
top,user_013,cereal,NaN,search
freq,213,80,NaN,4913
mean,NaN,NaN,1.719666e+09,NaN
std,NaN,NaN,9.062495e+06,NaN
min,NaN,NaN,1.704046e+09,NaN
25%,NaN,NaN,1.711964e+09,NaN
50%,NaN,NaN,1.719433e+09,NaN
75%,NaN,NaN,1.727557e+09,NaN


### データセットグループの作成

In [6]:
personalize = boto3.client(
    'personalize',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

personalize_runtime = boto3.client(
    'personalize-runtime', 
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

create_dataset_group_response = personalize.create_dataset_group(
    name = DATASET_GROUP_NAME,
)

dataset_group = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation",
  "ResponseMetadata": {
    "RequestId": "05414370-9f4b-4229-9a8a-29836a1c6024",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 23:13:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "111",
      "connection": "keep-alive",
      "x-amzn-requestid": "05414370-9f4b-4229-9a8a-29836a1c6024",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### max_time を上限として、データセットグループの作成が完了したことを確認し続ける

In [7]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    if status == "ACTIVE":
        print(f"Dataset group create succeeded for {dataset_group}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_group}")
        break
                
    time.sleep(10)

Dataset group create succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation


### インタラクションデータセットのスキーマとデータセットを作成する

In [8]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}

### すでに Schema を作っていたら削除、なければ作成

In [13]:
# スキーマの一覧を取得
response = personalize.list_schemas()
schema_arn = None

for schema in response['schemas']:
    if schema['name'] == SCHEMA_NAME:
        schema_arn = schema['schemaArn']
        break

if schema_arn:
    personalize.delete_schema(
        schemaArn=schema_arn
    )
    
    sleep(5)
        
    create_schema_response = personalize.create_schema(
        name = SCHEMA_NAME,
        schema = json.dumps(interactions_schema)
    )

else:
    create_schema_response = personalize.create_schema(
        name = SCHEMA_NAME,
        schema = json.dumps(interactions_schema)
    )

In [14]:
# スキーマの中身を確認
interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-1:627614495110:schema/suggest-word-1",
  "ResponseMetadata": {
    "RequestId": "50cc86e1-ce85-4b19-abe1-4855324604ea",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 23:26:24 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "85",
      "connection": "keep-alive",
      "x-amzn-requestid": "50cc86e1-ce85-4b19-abe1-4855324604ea",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 先ほど作成したスキーマを指定して、データセットグループにInteractionsデータセットを作成する

In [15]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group,
    schemaArn = interaction_schema_arn
)

interactions_dataset = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset/word_suggest_recommendation/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "589c442b-10d2-4f06-a7c8-a4137238113d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 23:26:24 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "589c442b-10d2-4f06-a7c8-a4137238113d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    if status == "ACTIVE":
        print(f"Dataset import succeeded for {interactions_dataset}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {interactions_dataset}")
        break
                
    time.sleep(60)

### S3でインタラクションCSVをステージングする
### 先ほど作成したInteractions CSVを先ほど作成したPersonalizeデータセットにアップロードする前に、S3バケットにCSVをステージングする必要がある
### S3バケットを作成し、Interactions CSVファイルをバケットにコピーしよう。

In [16]:
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,    
)

ACCOUNT_ID = boto3.client(
    'sts',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).get_caller_identity().get('Account')

BUCKET_NAME = f"amazon-pantry-personalize-example-{ACCOUNT_ID}"

print(BUCKET_NAME)

if REGION_NAME == "us-east-1":
    s3.create_bucket(Bucket=BUCKET_NAME)
else:
    s3.create_bucket(
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION_NAME}
    )

amazon-pantry-personalize-example-627614495110


### S3 へのアップロード

In [17]:
boto3.Session().resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).Bucket(BUCKET_NAME).Object(
    INTERACTIONS_FILENAME).upload_file(
    INTERACTIONS_FILENAME)

### S3バケットポリシーとIAMロールの作成
### データセットのインポートジョブをPersonalizeに送信する前に、Personalizeにバケットへのアクセス権を与えるバケットポリシーとIAMロールを作成する必要があります。

In [18]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{BUCKET_NAME}",
                f"arn:aws:s3:::{BUCKET_NAME}/*",
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=BUCKET_NAME, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'D3W6A2EYJ4TH1EHA',
  'HostId': '0T++CpeCNmWo820XBNqaDj9/rV/k8k3rNwizQVYVvc0Ba6T69zuRvpitr/V8lV2qGqWjm8SI/w8=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '0T++CpeCNmWo820XBNqaDj9/rV/k8k3rNwizQVYVvc0Ba6T69zuRvpitr/V8lV2qGqWjm8SI/w8=',
   'x-amz-request-id': 'D3W6A2EYJ4TH1EHA',
   'date': 'Sat, 31 May 2025 23:26:28 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [19]:
iam = boto3.client(
    "iam",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = ROLE_NAME,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )
except:
    create_role_response = iam.get_role(RoleName=ROLE_NAME)

In [20]:
role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::627614495110:role/PersonalizeRoleAmazonPantryAwsSimilarItems


### インタラクションデータセットのインポート
### S3バケットにあるステージングされたInteractions CSVを、作成したPersonalizeデータセットにインポートする準備を行う

In [22]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-interactions-import",
    datasetArn = interactions_dataset,
    dataSource = {
        "dataLocation": f"s3://{BUCKET_NAME}/{INTERACTIONS_FILENAME}"
    },
    roleArn = role_arn
)

dataset_import_job = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import",
  "ResponseMetadata": {
    "RequestId": "7f2cd3f5-ca76-4927-9ea8-61778e257a3d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 23:29:26 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "7f2cd3f5-ca76-4927-9ea8-61778e257a3d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 相互作用データセットのインポートジョブの完了を待つ
### 次のセルは、両方のインポート・ジョブが完了するまで待機します。

In [23]:
%%time

in_progress_import_arns = [ dataset_import_job ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    if status == "ACTIVE":
        print(f"Dataset import succeeded for {dataset_import_job}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_import_job}")
        break
                
    time.sleep(60)

Dataset import succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import
CPU times: user 184 ms, sys: 12.5 ms, total: 197 ms
Wall time: 4min 1s


### ソリューションとソリューションバージョンの作成
### 各データセットグループのデータのユーザパーソナライズのレシピを使って、ソリューションとソリューションのバージョンを作成します。
### まず、利用可能なパーソナライズレシピを列挙します。

In [24]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-ecomm-customers-who-viewed-x-also-viewed',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2024, 6, 21, 21, 49, 44, 731000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-frequently-bought-together',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2024, 6, 21, 21, 49, 44, 731000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-popular-items-by-purchases',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdat

### レシピの選択

### まず、各レシピのソリューションとソリューション・バージョンを作成する。

In [25]:
sims_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-sims-solution-example",
    datasetGroupArn = dataset_group,
    recipeArn = SIMS_RECIPE_ARN
)

sims_solution_arn = sims_create_solution_response['solutionArn']

In [26]:
sims_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [27]:
sims_solution_version_arn = sims_solution_version_response['solutionVersionArn']
print(json.dumps(sims_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/b6b2f582",
  "ResponseMetadata": {
    "RequestId": "a02d00c5-6f80-4b67-ad1e-6c3455661e27",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 23:33:43 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "a02d00c5-6f80-4b67-ad1e-6c3455661e27",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### ソリューション・バージョンがアクティブになるのを待つ
### 最後に、ソリューションバージョンが作成され終わるのを待ちます。このステップでPersonalizeはデータセットと選択されたレシピに基づいて機械学習モデルをトレーニングします。
### また、Personalizeはインタラクションのデータセットをトレーニング部分と評価部分に分割し、トレーニング済みのモデルに対するレコメンデーションの品質を評価できるようにします。
### 説明データを含むデータセットグループのソリューションバージョンは、説明データを含まないものよりもトレーニングに時間がかかることがわかります

In [28]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    version_response = personalize.describe_solution_version(
        solutionVersionArn = sims_solution_version_arn
    )
    status = version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE":
        print(f"Build succeeded for {sims_solution_version_arn}")
        break
    elif status == "CREATE FAILED":
        print(f"Build failed for {sims_solution_version_arn}")
        break
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/b6b2f582
CPU times: user 539 ms, sys: 53.4 ms, total: 592 ms
Wall time: 13min 7s


### 処理時間

In [29]:
response = personalize.describe_solution_version(solutionVersionArn = sims_solution_version_arn)
training_hours_sims = response['solutionVersion']['trainingHours']
print(training_hours_sims)

0.787


### キャンペーンの作成

In [30]:
sims_solution = {
        "solution_arn": sims_solution_arn,
        "solution_version_arn": sims_solution_version_arn
}

In [31]:
create_campaign_response = personalize.create_campaign(
    name = f"personalize-demo-sims-example",
    solutionVersionArn = sims_solution['solution_version_arn'],
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:' + campaign_arn)

campaign_arn:arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example


In [32]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print(f"Campaign: {campaign_arn} {status}")

    if status == "ACTIVE" or status == "CREATE FAILED":
        break

    time.sleep(60)

Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE PENDING
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personali

### 推薦実行

In [34]:
search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,user_001,coffee,1704067200,search
1,user_001,tea,1704067200,search
2,user_001,milk,1704067200,search
3,user_002,bread,1704153600,search
4,user_002,cheese,1704153600,search


In [35]:
search_log_count_df = pd.DataFrame(
    {
        "ITEM_ID": search_log_df['ITEM_ID'].value_counts().index,
        "COUNT": search_log_df['ITEM_ID'].value_counts().values,
    }
)

search_log_count_df.head()

,ITEM_ID,COUNT
0,cereal,80
1,honey,75
2,mushroom,70
3,fork,70
4,plate,70


In [36]:
search_query = search_log_count_df.sample()["ITEM_ID"].item()
print(search_query)

cleaner


In [37]:
search_log_count_df.query(f'ITEM_ID=="{search_query}"')

,ITEM_ID,COUNT
24,cleaner,62


In [38]:
# 与えるデータ
recommended_item_df = search_log_count_df.query(f'ITEM_ID=="{search_query}"')
recommended_item_df

,ITEM_ID,COUNT
24,cleaner,62


In [45]:
### search_query のワードが使われているTIMESTAMP
cleaner_timestamps = search_log_df.loc[search_log_df['ITEM_ID'] == search_query, 'TIMESTAMP'].unique()
cleaner_timestamps

array([1706227200, 1709164800, 1712016000, 1714953600, 1717891200,
       1720742400, 1723593600, 1726531200, 1729468800, 1732320000,
       1715053454, 1723351169, 1727489950, 1705510221, 1707133721,
       1733768335, 1719280526, 1719118012, 1735111782, 1720640373,
       1710550127, 1729220264, 1713156787, 1720558814, 1725262643,
       1726198762, 1722007329, 1728961989, 1726836882, 1717566789,
       1729862113, 1735257237, 1735554514, 1713447261, 1710684828,
       1733907998, 1734607103, 1728824224, 1723657479, 1728299261,
       1709445065, 1706243446, 1715791729, 1705474135, 1730439834,
       1706575839, 1717220859, 1708842223, 1724223227, 1722150871,
       1726143662, 1726057631, 1727272391, 1733656892, 1715135868,
       1723961071, 1733396880, 1704702838, 1731283982, 1707709283,
       1709408425, 1729523137])

In [53]:
co_searched_df = search_log_df[search_log_df['TIMESTAMP'].isin(cleaner_timestamps)]
co_searched_word = co_searched_df.loc[search_log_df['ITEM_ID'] != search_query, 'ITEM_ID'].unique()
sorted(co_searched_word)

['bag',
 'bean',
 'beef',
 'beer',
 'bleach',
 'blueberry',
 'bottle',
 'bowl',
 'box',
 'bread',
 'broccoli',
 'brush',
 'burger',
 'butter',
 'cake',
 'can',
 'carrot',
 'cereal',
 'cheese',
 'chocolate',
 'cookie',
 'corn',
 'cup',
 'detergent',
 'egg',
 'fish',
 'flour',
 'fork',
 'garlic',
 'ginger',
 'grape',
 'herb',
 'jam',
 'jar',
 'knife',
 'lemon',
 'mango',
 'mint',
 'mushroom',
 'onion',
 'orange',
 'pasta',
 'pea',
 'pepper',
 'pineapple',
 'plate',
 'potato',
 'salad',
 'salt',
 'sandwich',
 'sauce',
 'shampoo',
 'soap',
 'soda',
 'spice',
 'spinach',
 'sponge',
 'sugar',
 'tissue',
 'tomato',
 'wine',
 'yogurt']

In [55]:
co_searched_items = co_searched_df[co_searched_df['ITEM_ID'] != search_query]
co_searched_items['ITEM_ID'].value_counts()

ITEM_ID
bleach      7
brush       6
sauce       6
mushroom    5
mint        4
           ..
salad       1
bag         1
broccoli    1
ginger      1
jam         1
Name: count, Length: 62, dtype: int64

### レコメンド

In [39]:
response = personalize_runtime.get_recommendations(
    campaignArn=campaign_arn,
    itemId=search_query,
    numResults=10
)
print(response['itemList'])

[{'itemId': 'mango'}, {'itemId': 'shampoo'}, {'itemId': 'mint'}, {'itemId': 'spinach'}, {'itemId': 'pepper'}, {'itemId': 'bottle'}, {'itemId': 'onion'}, {'itemId': 'tissue'}, {'itemId': 'toothpaste'}, {'itemId': 'butter'}]


In [40]:
# 与えたデータを元に推薦実行
recommendation_df = pd.DataFrame(
    [dic['itemId'] for dic in response['itemList']],
    columns=["ITEM_ID"]
)

recommendation_df

,ITEM_ID
0,mango
1,shampoo
2,mint
3,spinach
4,pepper
5,bottle
6,onion
7,tissue
8,toothpaste
9,butter


### リソースの削除

In [56]:
# personalize.delete_campaign(campaignArn = campaign_arn)

{'ResponseMetadata': {'RequestId': 'e48949c2-d7d5-42cf-9442-6902729d3971',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 01 Jun 2025 00:14:15 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e48949c2-d7d5-42cf-9442-6902729d3971',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'cache-control': 'no-cache',
   'x-content-type-options': 'nosniff'},
  'RetryAttempts': 0}}